In [1]:
import requests
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
from datetime import datetime
import numpy as np
from googlesearch import search
import re

In [2]:
def fghtdta(name):
    
    #name="tony ferguson"
    query1 = "wikipedia"+ name
    query2 = "ufcstats com"+ name
    query3 = "fightmatrix"+ name
    
    urls1=[]
    urls2=[] 
    urls3=[] 
    
    for j in search(query1, tld="co.in", num=10, stop=1, pause=2): 
        urls1.append(j)
    for j in search(query2, tld="co.in", num=10, stop=1, pause=2): 
        urls2.append(j)
    for j in search(query3, tld="co.in", num=10, stop=1, pause=2): 
        urls3.append(j)
        
    url1 = urls1[0]
    url2 = urls2[0]
    url3 = urls3[0]
    
    req1 = urllib.request.urlopen(url1)
    article1 = req1.read().decode()
    soup1 = BeautifulSoup(article1, 'html.parser')
    
    req2 = urllib.request.urlopen(url2)
    article2 = req2.read().decode()
    soup2 = BeautifulSoup(article2, 'html.parser')
    
    
    req3 = urllib.request.urlopen(url3)
    article3 = req3.read().decode()
    soup3 = BeautifulSoup(article3, 'html.parser')
    tables = soup1.find_all('table', class_='wikitable')
    tr=tables[1].find_all('tr')
    X1= []
    X2= []
    X3= []
    X4= []
    X5= []
    X6= []
    X7= []
    for items in tr[1:]:
        info=items.find_all('td')
        X1.append(info[0].text.strip())
        X2.append(info[1].text.strip())
        X3.append(info[2].text.strip())
        X4.append(info[3].text.strip())
        X5.append(info[4].text.strip())
        X6.append(info[5].text.strip())
        X7.append(info[6].text.strip())
    tab1= pd.DataFrame (list(zip(X1,X2,X3,X4,X5,X6,X7)))
    index=[]
    for i in range(7):
        index.append(tr[0].find_all('th')[i].text.strip())
    tab1.columns=index
    ageatfight=[]
    for date in tab1["Date"]:
        ageatfight.append((datetime.strptime(date, '%B %d, %Y')-datetime.strptime(soup1.find_all('span', class_="bday")[0].text.strip(), '%Y-%m-%d')).days/365)
    tab1["ageatfight"]=ageatfight
    timeout=[0]
    for i in np.flip(range(len(tab1)))[:-1]:
            timeout.append((datetime.strptime(tab1["Date"][i-1], '%B %d, %Y')-datetime.strptime(tab1["Date"][i], '%B %d, %Y')).days/365)
    tab1["timeout"]=np.flip(timeout)
    tab1["Round"]= list(map(int, tab1["Round"])) 
    cumrnds=np.cumsum(tab1["Round"],axis=0) 
    cumrnds=np.flip(cumrnds, axis=0).values
    tab1["cumrnds"]=cumrnds
    stats=soup2.find_all('li')
    X8 = []
    X9 = []
    for items in stats[3:16]:
        X8.append(items.text.strip()[-8:])
        X9.append(items.text.strip()[:5])
    for i in range (len(X8)):
        tab1[X9[i]]=np.repeat(X8[i],len(tab1))
    stats2=soup3.find_all('td', class_="tdRank")[1].find_all('div')
    X10 = []
    X11 = []
    for items in stats2:
        X11.append(items.text.strip().split(':')[0])
        X10.append(items.text.strip().split(':')[1])
    for i in range (len(X10)):
        tab1[X11[i]]=np.repeat(X10[i],len(tab1))
    X12 = []
    X13 = []
    for rec in tab1['Record']:
        X12.append(rec.split('–')[0])
        X13.append(rec.split('–')[1])
    X14 = []
    X15 = []
    for rec in tab1["'Big League' Record"]:
        X14.append(rec.split('-')[0])
        X15.append(rec.split('-')[1])
    X16 = []
    X17 = []
    for rec in tab1["Last 3 Years"]:
        X16.append(rec.split('-')[0])
        X17.append(rec.split('-')[1])
    X18 = []
    for i in range(len(tab1['Last Quality Perf.'])):
        X18.append((datetime.strptime(tab1['Date'][i], '%B %d, %Y')-datetime.strptime(tab1['Last Quality Perf.'][i][1:], '%Y-%m-%d')).days>180)
    tab1['W1']=[re.sub("[\(\[].*?[\)\]]", "", value) for value in X12]
    tab1['W2']=[re.sub("[\(\[].*?[\)\]]", "", value) for value in X13]
    tab1['W3']=[re.sub("[\(\[].*?[\)\]]", "", value) for value in X14]
    tab1['L1']=[re.sub("[\(\[].*?[\)\]]", "", value) for value in X15]
    tab1['L2']=[re.sub("[\(\[].*?[\)\]]", "", value) for value in X16]
    tab1['L3']=[re.sub("[\(\[].*?[\)\]]", "", value) for value in X17]
    tab1['pstprm']=[value*1 for value in X18]
    alphanumeric = [character for character in tab1['Heigh'][0] if character.isalnum()]
    tab1['Heigh']=int(alphanumeric[0])*30.48+int("".join(alphanumeric[1:]))*2.54
    tab1['Win Finish %']=[float(value[:-1]) for value in tab1['Win Finish %']]
    tab1['Str. ']=[float(value[:-1]) for value in tab1['Str. ']]
    tab1['TD Ac']=[float(value[:-1]) for value in tab1['TD Ac']]
    tab1['TD De']=[float(value[:-1]) for value in tab1['TD De']]
    tab1['Quality Perf. %']=[float(value[:-1]) for value in tab1['Quality Perf. %']]
    tab1['Reach']=[float(value[:-1])*2.54 for value in tab1['Reach']]
    tab1['Ape']=tab1['Reach']/tab1['Heigh']
    tab1['Weigh']=[float(value[:4]) for value in tab1['Weigh']]
    tab1['Record'] = tab1['Record'].shift(-1)
    tab1['ageatfight'] = tab1['ageatfight'].shift(-1)
    tab1['cumrnds'] = tab1['cumrnds'].shift(-1)
    tab1['W1']=tab1['W1'].shift(-1)
    tab1['W2']=tab1['W2'].shift(-1)
    tab1['W3']=tab1['W3'].shift(-1)
    tab1['L1']=tab1['L1'].shift(-1)
    tab1['L2']=tab1['L2'].shift(-1)
    tab1['L3']=tab1['L3'].shift(-1)
    return tab1

In [3]:
tony=fghtdta("tony ferguson")

In [4]:
tony

,Res.,Record,Opponent,Method,Event,Date,Round,ageatfight,timeout,cumrnds,...,540 Metric,Rating Points,W1,W2,W3,L1,L2,L3,pstprm,Ape
0,Win,24–3,Donald Cerrone,TKO (doctor stoppage),UFC 238,"June 8, 2019",2,34.671233,0.671233,56.0,...,.650,1394,24,3,15,1,3,0,0,1.070423
1,Win,23–3,Anthony Pettis,TKO (corner stoppage),UFC 229,"October 6, 2018",2,33.673973,0.997260,54.0,...,.650,1394,23,3,15,1,3,0,0,1.070423
2,Win,22–3,Kevin Lee,Submission (triangle choke),UFC 216,"October 7, 2017",3,32.753425,0.920548,52.0,...,.650,1394,22,3,15,1,3,0,0,1.070423
3,Win,21–3,Rafael dos Anjos,Decision (unanimous),The Ultimate Fighter Latin America 3 Finale: d...,"November 5, 2016",5,32.438356,0.315068,51.0,...,.650,1394,21,3,15,1,3,0,0,1.070423
4,Win,20–3,Lando Vannata,Submission (D'Arce choke),UFC Fight Night: McDonald vs. Lineker,"July 13, 2016",2,31.849315,0.589041,48.0,...,.650,1394,20,3,15,1,3,0,0,1.070423
5,Win,19–3,Edson Barboza,Submission (D'Arce choke),The Ultimate Fighter: Team McGregor vs. Team F...,"December 11, 2015",2,31.441096,0.408219,47.0,...,.650,1394,19,3,15,1,3,0,0,1.070423
6,Win,18–3,Josh Thomson,Decision (unanimous),UFC Fight Night: Mir vs. Duffee,"July 15, 2015",3,31.065753,0.375342,46.0,...,.650,1394,18,3,15,1,3,0,0,1.070423
7,Win,17–3,Gleison Tibau,Submission (rear-naked choke),UFC 184,"February 28, 2015",1,30.835616,0.230137,43.0,...,.650,1394,17,3,15,1,3,0,0,1.070423
8,Win,16–3,Abel Trujillo,Submission (rear-naked choke),UFC 181,"December 6, 2014",2,30.567123,0.268493,42.0,...,.650,1394,16,3,15,1,3,0,0,1.070423
9,Win,15–3,Danny Castillo,Decision (split),UFC 177,"August 30, 2014",3,30.298630,0.268493,41.0,...,.650,1394,15,3,15,1,3,0,0,1.070423


In [5]:
Cerrone=fghtdta("donald cerrone")

In [6]:
Cerrone.iloc[:,:]

,Res.,Record,Opponent,Method,Event,Date,Round,ageatfight,timeout,cumrnds,...,540 Metric,Rating Points,W1,W2,W3,L1,L2,L3,pstprm,Ape
0,Loss,36–13 (1),Conor McGregor,TKO (head kick and punches),UFC 246,"January 18, 2020",1,36.487671,0.345205,105.0,...,.629,416,36,13,30,14,4,6,1,1.0
1,Loss,36–12 (1),Justin Gaethje,TKO (punches),UFC Fight Night: Cowboy vs. Gaethje,"September 14, 2019",1,36.219178,0.268493,103.0,...,.629,416,36,12,30,14,4,6,0,1.0
2,Loss,36–11 (1),Tony Ferguson,TKO (doctor stoppage),UFC 238,"June 8, 2019",2,36.123288,0.095890,102.0,...,.629,416,36,11,30,14,4,6,0,1.0
3,Win,35–11 (1),Al Iaquinta,Decision (unanimous),UFC Fight Night: Iaquinta vs. Cowboy,"May 4, 2019",5,35.835616,0.287671,101.0,...,.629,416,35,11,30,14,4,6,0,1.0
4,Win,34–11 (1),Alexander Hernandez,TKO (head kick and punches),UFC Fight Night: Cejudo vs. Dillashaw,"January 19, 2019",2,35.643836,0.191781,100.0,...,.629,416,34,11,30,14,4,6,0,1.0
5,Win,33–11 (1),Mike Perry,Submission (armbar),UFC Fight Night: Korean Zombie vs. Rodríguez,"November 10, 2018",1,35.260274,0.383562,99.0,...,.629,416,33,11,30,14,4,6,0,1.0
6,Loss,33–10 (1),Leon Edwards,Decision (unanimous),UFC Fight Night: Cowboy vs. Edwards,"June 23, 2018",5,34.917808,0.342466,97.0,...,.629,416,33,10,30,14,4,6,0,1.0
7,Win,32–10 (1),Yancy Medeiros,TKO (punches),UFC Fight Night: Cowboy vs. Medeiros,"February 18, 2018",1,34.589041,0.328767,96.0,...,.629,416,32,10,30,14,4,6,0,1.0
8,Loss,32–9 (1),Darren Till,TKO (punches),UFC Fight Night: Cowboy vs. Till,"October 21, 2017",1,34.358904,0.230137,95.0,...,.629,416,32,9,30,14,4,6,0,1.0
9,Loss,32–8 (1),Robbie Lawler,Decision (unanimous),UFC 214,"July 29, 2017",3,33.860274,0.498630,92.0,...,.629,416,32,8,30,14,4,6,0,1.0
